In [ ]:
#6-Building Advanced RAG Q&A Project With Multiple Data Sources With Langchain

'''
1. Tools -> Toolkits
2. Agents
Use ARXIV, Wiki, Pdfs as a separate tool and wrap all this in the form of Toolkit and 
with the help of agents we can execute any Q&A seacrh
'''



'\n1. Tools -> Toolkits\n2. Agents\nUse ARXN, Wiki, Pdfs as a separate tool and wrap all this in the form of Toolkit and \nwith the help of agents we can execute any Q&A seacrh\n'

![alt text](<Screenshot (223).png>)

In [3]:
# Wrapper of arxiv and wiki
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [4]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [5]:
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\ps1477\\Documents\\projects\\AI-study\\RAG\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200))

In [6]:
wiki.name #tool 1

'wikipedia'

In [7]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.
c:\Users\ps1477\Documents\projects\AI-study\RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001")
vectordb = FAISS.from_documents(documents,embeddings)
retriever = vectordb.as_retriever()
retriever

SSLError: HTTPSConnectionPool(host='docs.smith.langchain.com', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:992)')))

In [ ]:
from langchain.tools.retriever import create_retriever_tool
#below tool to do search specifically for this page - langsmith
retriever_tool = create_retriever_tool(retriever, "langsmith_search", "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [ ]:
retriever_tool.name #2nd tool - own customized tool 

'langsmith_search'

In [ ]:
# Arxiv - Websited where all the research papers are uploaded (Arxiv tool is 3rd)
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

#create wrapper
arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)

In [ ]:
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>))

In [ ]:
arxiv.name #3rd tool

'arxiv'

In [ ]:
''' 
Arxiv and wiki - this two tools are given by langchain 
langsmith is we have created - custom tool
''' 
#now combine this three tools
tools=[wiki, arxiv, retriever_tool]


In [ ]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\ps1477\\Documents\\projects\\AI-study\\RAG\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200, arxiv_result=<class 'arxiv.Result'>)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001A102520540>, retriever=VectorStoreRetriever(tags=['FAISS', '

In [ ]:
from langchain_groq import ChatGroq
import os

groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(temperature=0.6, groq_api_key=groq_api_key, model_name="gemma-7b-it")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001A14E881F90>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001A14E883A90>, model_name='gemma-7b-it', temperature=0.6, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain import hub  #by this we will get pre-written prompts from lancghian - noneed to create own (can write own too if need)
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [ ]:
#now query from this tools so to do this we use agents 
### Agents - can use agents of open source models too like gemini
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [ ]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)  #,verbose=True -> for detailed op
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [ ]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Skip to main contentGo to API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​We provide multiple ways to log traces to

{'input': 'Tell me about Langsmith',
 'output': "The provided text does not contain any information regarding Langsmith's capabilities or outputs related to the given query. Therefore, I am unable to extract the requested data from the provided context."}

In [ ]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper 1605.08386 is about **Heat-bath random walks on graphs**. It was published in 2016 and authored by Caprice Stanley and Tobias Windisch.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper 1605.08386 is about **Heat-bath random walks on graphs**. It was published in 2016 and authored by Caprice Stanley and Tobias Windisch.'}

In [ ]:
agent_executor.invoke({"input":"What is machine learning?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and gen...

> Finished chain.


{'input': 'What is machine learning?', 'output': '...'}

In [ ]:
agent_executor.invoke({"input":"What is current temperature of bangalore?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'temperature bangalore'}`


Page: Bangalore
Summary: Bangalore ( BANG-gə-lor, -⁠LOR), officially Bengaluru (Kannada pronunciation: [ˈbeŋgɐɭuːɾu] ; ISO: Beṁgaḷūru), is the capital and largest city of the southern Indian state of The current temperature in Bangalore is not provided in the given context.

> Finished chain.


{'input': 'What is current temperature of bangalore?',
 'output': 'The current temperature in Bangalore is not provided in the given context.'}